### Importing the nucleotide sequences I am using and adding them to a string container.

In [5]:
def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

input_name = 'Forrest B100B10 BAC sequence Rhg4.txt'
forrest_sequence = read_file(input_name)
print(f'First 50 nucleotides of Forrest sequence: {forrest_sequence[:50]}')

First 50 nucleotides of Forrest sequence: CCCTCGCGAGTTGGTTCAGCTGCTGCCTGAGGCTGGACGACCTCGCGGAG


In [19]:
# def read_file(input_name):
#     sequence = ''
#     with open(input_name) as input_file:
#         input_file.readline()  # skip header
        
#         seq = input_file.readline().strip()
#         while seq:
#             sequence += seq
#             seq = input_file.readline().strip()
#     return sequence

# input_name = 'Forrest B100B10 BAC sequence Rhg4.txt'
# forrest_sequence = read_file(input_name)
# print(f'First 50 nucleotides of Forrest sequence: {forrest_sequence[:50]}')

First 50 nucleotides of Forrest sequence: CCCTCGCGAGTTGGTTCAGCTGCTGCCTGAGGCTGGACGACCTCGCGGAG


In [18]:
s = ''
if s:
    print('True')
else:
    print('False')

False


In [18]:
input_name = 'Forrest SHMT.txt'
shmt = read_file(input_name)
print(f'First 50 nucleotides of SHMT gene sequence: {shmt[:50]}')

First 50 nucleotides of SHMT gene sequence: CAATGGCACCAATGCCCAATGGGAGATTTAAGTCAAGCCCAACATCAACC


#### From the results of my thesis, I know the boundaries of the genes in the Rhg4 region of interest. Moving from one end of the region to the other, trying to locate the genes using Python.

In [7]:
# finding gene boundaries
def find_start_end(gene, variant):
    # using find to find the start index
    # start index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(gene)
    while index >= 0:
        indices.append((index+1, index+len(gene)))  # bp locations of genes reported by databases start from 1
        count += 1
        if count % 10:
            print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(gene, index+len(gene))
    return indices

# variant = forrest_sequence
# gene = shtm
variant = 'CCCTCGCAGTTGG'
gene = 'TCG'
boundary_indices = find_start_end(gene, variant)
print(f'boundary_indices: {boundary_indices}')

found at index 3, len(variant): 13
boundary_indices: [(4, 6)]


In [11]:
# finding gene boundaries
def find_start_end(gene, variant):
    # using find to find the start index
    # start index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(gene)
    while index >= 0:
        indices.append((index+1, index+len(gene)))  # bp locations of genes reported by databases start from 1
        count += 1
        if count % 10:
            print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(gene, index+len(gene))
    return indices

variant = forrest_sequence
gene = shtm
print(f'len(variant): {len(variant)}, len(gene): {len(gene)}')
# variant = 'CCCTCGCAGTTGG'
# gene = 'TCG'
boundary_indices = find_start_end(gene, variant)
print(f'boundary_indices: {boundary_indices}')

len(variant): 94336, len(gene): 5103
boundary_indices: []


In [4]:
# finding gene boundaries
def find_start_end(gene, variant):
    # using find to find the start index
    # start index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(gene)
    while index >= 0:
        indices.append((index+1, index+len(gene)))  # bp locations of genes reported by databases start from 1
        count += 1
        if count % 10:
            print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(gene, index+len(gene))
    return indices

variant = forrest_sequence
gene = shmt[2000:2005]
print(f'len(variant): {len(variant)}, len(gene): {len(gene)}')
# variant = 'CCCTCGCAGTTGG'
# gene = 'TCG'
boundary_indices = find_start_end(gene, variant)
print(f'boundary_indices: {boundary_indices}')

len(variant): 94336, len(gene): 1316
boundary_indices: []


### At this point, I ran into problem of Python not finding the gene sequence in Forrest sequence that we know should be there. I am taking the approach of comparing the Forrest sequence with the gene sequence using other bioinformatics tools.

#### I used BLAST (alignment tool) to zone in on the coding region of SHTM gene (while comparing sequences in BLAST). Adjusting the subsequence we're searching for should give us the result we're expecitng. The reason we don't is that the gene sequence we're searching for is oriented the wrong way.

In [4]:
def find_start_end(gene, variant):
    # using find to find the start index
    # start index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(gene)
    while index >= 0:
        indices.append((index+1, index+len(gene)))  # bp locations of genes reported by databases start from 1
        count += 1
        if count % 10:
            print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(gene, index+len(gene))
    return indices

variant = forrest_sequence
gene = shmt[2339:3657]
print(f'len(variant): {len(variant)}, len(gene): {len(gene)}')
# variant = 'CCCTCGCAGTTGG'
# gene = 'TCG'
boundary_indices = find_start_end(gene, variant)
print(f'boundary_indices: {boundary_indices}')

len(variant): 94336, len(gene): 1318
boundary_indices: []


#### The Gene sequence (Forrest SHMT coding sequence) we're searching for is oriented the wrong way in relation to the Variant sequence. Need to generate a reverse complement of the gene sequence before running the find boundaries function.

In [14]:
def complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter]
    return complement_seq

sequence = 'AAAACCCGGT'
complement = complement(sequence)
print(f'complement of {sequence}: {complement}')

complement of AAAACCCGGT: TTTTGGGCCA


In [26]:
def reverse_complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter]
    return complement_seq[::-1]

def complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter]
    return complement_seq

# -         21 987654321
#           0123456789 1
sequence = 'AGCTGCTGCCTG'
print(f'original sequence:   {sequence}')
complement_sequence = complement(sequence)
print(f'complement sequence: {complement_sequence}')
reverse_complement = reverse_complement(sequence)
print(f'reverse complement:  {reverse_complement}')

print(f'sequence left to right: {sequence[0:12]}')
print(f'sequence right to left: {sequence[-1:-13:-1]}')

print(f'middle 8 characters left to right: {sequence[2:10]}')
print(f'middle 8 characters right to left: {sequence[-3:-11:-1]}')

original sequence:   AGCTGCTGCCTG
complement sequence: TCGACGACGGAC
reverse complement:  CAGGCAGCAGCT
sequence left to right: AGCTGCTGCCTG
sequence right to left: GTCCGTCGTCGA
middle 8 characters left to right: CTGCTGCC
middle 8 characters right to left: CCGTCGTC


In [4]:
def reverse_complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter]
    return complement_seq[::-1]

def find_start_end(gene, variant):
    # using find to find the start index
    # start index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(gene)
    # print(f'index before entering while loop: {index}')
    while index >= 0:
        # print(f'entered while loop')
        indices.append((index+1, index+len(gene)+1))  # bp locations of genes reported by databases start from 1
        count += 1
        if count % 10:
            print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(gene, index + len(gene))
    return indices

sequence = shmt[2340:3657]
variant = forrest_sequence
gene = reverse_complement(sequence)
boundary_indices = find_start_end(gene, variant)
print(f'boundary_indices: {boundary_indices}')

found at index 1439, len(variant): 94336
boundary_indices: [(1440, 2757)]


#### Found the Forrest Serine hydroxymethyltransferase (SHMT) gene. It is where we expected it to be in the Forrest sequence.

### Performing the searches for start and for stop codons to confirm the location of the coding sequence of Forrest SHMT. We also know that the first index returned by the find_start_end function has to match up with the first index found by the find_start_codon function.

In [1]:
# why ATG as the codon?

def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

# finding start codon
def find_start_codon(codon, variant, offset):
    # using find to find the codon index
    # codon index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(codon)
    while index >= 0:
        indices.append(index+offset+1)  # bp locations of genes reported by databases codon from 1
        count += 1
        # if count % 10:
        #     print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(codon, index+len(codon))
    return indices

input_name = 'Forrest B100B10 BAC sequence Rhg4.txt'
codon = 'ATG'
variant = read_file(input_name)
offset = 1439
ATG_codon_indices = find_start_codon(codon, variant, offset)
print(f'length of ATG codon indices: {len(ATG_codon_indices)}, ATG_codon_indices[:10]: {ATG_codon_indices[:10]}')

length of ATG codon indices: 1882, ATG_codon_indices[:10]: [1551, 1583, 1710, 1790, 1865, 1927, 1992, 2027, 2131, 2157]


In [1]:
# other strand of DNA
def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

# finding start codon
def find_start_codon(codon, variant, offset):
    # using find to find the codon index
    # codon index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(codon)
    while index >= 0:
        indices.append(index+offset+1)  # bp locations of genes reported by databases codon from 1
        count += 1
        # if count % 10:
        #     print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(codon, index+len(codon))
    return indices

input_name = 'Forrest B100B10 BAC sequence Rhg4.txt'
codon = 'TAG'
variant = read_file(input_name)
offset = 1439
TAG_codon_indices = find_start_codon(codon, variant, offset)
print(f'length of ATG codon indices: {len(TAG_codon_indices)}, ATG_codon_indices[:10]: {TAG_codon_indices[:10]}')

length of ATG codon indices: 1236, ATG_codon_indices[:10]: [1725, 1904, 2023, 2281, 2293, 2482, 2619, 2701, 2818, 2824]


##### So far we're not finding the ATG start codon where it should be, but several things could be at play here. I am using both Python and bioinformatics tools to investigate.

#### Need for complement

In [7]:
def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

def reverse_complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter.upper()]
    return complement_seq[::-1]

# finding start codon
def find_start_codon(codon, variant, offset):
    # using find to find the codon index
    # codon index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(codon)
    while index >= 0:
        indices.append(offset+index+1)  # bp locations of genes reported by databases codon from 1
        count += 1
        # if count % 10:
        #     print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(codon, index+len(codon))
    return indices

input_name = 'Forrest B100B10 BAC sequence Rhg4.txt'
codon = reverse_complement('ATG')
print(f'codon: {codon}')
variant = read_file(input_name)
offset = 1439
start_codon_indices = find_start_codon(codon, variant[offset:2757+1], offset)
print(f'length of TAC codon indices: {len(start_codon_indices)}')
print(f'start_codon_indices[-1:-11:-1]: {start_codon_indices[-1:11:-1]}')

codon: CAT
length of TAC codon indices: 12
start_codon_indices[-1:-11:-1]: []


In [1]:
def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

def reverse_complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter.upper()]
    return complement_seq[::-1]

# finding start codon
def find_start_codon(codon, variant, offset):
    # using find to find the codon index
    # codon index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(codon)
    while index >= 0:
        indices.append(offset+index+1)  # bp locations of genes reported by databases codon from 1
        count += 1
        # if count % 10:
        #     print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(codon, index+len(codon))
    return indices

input_name = 'Forrest B100B10 BAC sequence Rhg4.txt'
codon = reverse_complement('ATG')
print(f'codon: {codon}')
variant = read_file(input_name)
offset = 1440
start_codon_indices = find_start_codon(codon, variant[offset:2757+1], offset)
print(f'length of TAC codon indices: {len(start_codon_indices)}')
print(f'TAC codon indices: {start_codon_indices}')
# print(f'start_codon_indices[-1:-11:-1]: {start_codon_indices[-1:11:-1]}')

codon: CAT
length of TAC codon indices: 12
TAC codon indices: [1467, 1636, 1768, 1938, 1999, 2110, 2383, 2541, 2569, 2666, 2697, 2755]


In [14]:
# accounting for the beginning of the codon being 2 position beyond the index returned

def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

def reverse_complement(sequence):
    complement_seq = ""
    complement_dict = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    for letter in sequence:
        complement_seq += complement_dict[letter.upper()]
    return complement_seq[::-1]

# finding start codon
def find_start_codon(codon, variant, offset):
    # using find to find the codon index
    # codon index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(codon)
    while index >= 0:
        indices.append(2+offset+index+1)  # databases count from 1, adding 2 for the actual start of the codon
        count += 1
        # if count % 10:
        #     print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(codon, index+len(codon))
    return indices

input_name = 'Forrest B100B10 BAC sequence Rhg4.txt'
codon = reverse_complement('ATG')
print(f'codon: {codon[::-1]}')
variant = read_file(input_name)
offset = 1440
start_codon_indices = find_start_codon(codon, variant[offset:2757+1], offset)
print(f'TAC codon indices: {start_codon_indices}')

codon: TAC
TAC codon indices: [1469, 1638, 1770, 1940, 2001, 2112, 2385, 2543, 2571, 2668, 2699, 2757]


In [ ]:
# finding the gene's stop codon recursively - not much use: not recursive by nature

def find_stop_codon_iter(codons, variant, offset):
    indices = []
    for codon in codons:
        index = find_stop_codon_recurisve(codon, variant, offset)
        if index >= 0:
            indices.append(index+1)  # plus 1 because biological databases start count from 1, not 0
    return indices

def find_stop_codon_recursive(codon, variant, offset):
    # breaking recursion: if indices for stop codon is -1
    index = variant.find(codon, offset)
    if index == -1:
        return index

codons = ['TAA', 'TAG', 'TGA']
variant = forrest_sequence
offset = 1440

In [12]:
# finding the gene's stop codon - iterative algorithm: finds stop codon where indeed it is

def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

def find_stop_codon(codons, variant, offset):
    indices = []
    for codon in codons:
        index = variant.find(codon, offset)
        while index >= 0:
            indices.append(index+1)  # plus 1 because biological databases start count from 1, not 0
            index = variant.find(codon, index+len(codon))
    return indices

input_name = 'Forrest B100B10 BAC sequence Rhg4.txt'
codons = ['TAA', 'TAG', 'TGA']
variant = read_file(input_name)
offset = 1439
stop_indices = find_stop_codon(codons, variant, offset)
print(f'indices of stop codons[:10]: {stop_indices[:10]}')
print(f'length of stop_indices: {len(stop_indices)}')
print(f'stop codon at 1445: {variant[1444:1441:-1]}')

indices of stop codons[:10]: [1445, 1471, 1482, 1551, 1557, 1642, 1803, 1824, 1920, 1940]
length of stop_indices: 6009
stop codon at 1445: TGA


### Found gene and its start and stop codons where they are. Do the sequences from the susceptible cultivars have the Forrest SHMT gene?

In [16]:
def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

input_name = 'Williams 82 BAC 56G2 sequence.txt'
williams_sequence = read_file(input_name)
print(f'First 50 nucleotides of Williams sequence: {williams_sequence[:50]}')

First 50 nucleotides of Williams sequence: AAGCTTGCATGCTTTCTGGTTTCCTTGCCAATCACTGCATGGAACAATCT


In [17]:
def read_file(input_name):
    sequence = ''
    with open(input_name) as input_file:
        input_file.readline()
        while True:
            seq = input_file.readline().strip()
            if len(seq) == 0:
                break
            sequence += seq
    return sequence

input_name = 'Asgrow sequence.txt'
asgrow_sequence = read_file(input_name)
print(f'First 50 nucleotides of Asgrow sequence: {asgrow_sequence[:50]}')

First 50 nucleotides of Asgrow sequence: AATCCCTCTTTAGTTAGCTAGTAACATCTCCCTTGTTCTTATATATAATC


In [19]:
def find_start_end(gene, variant):
    # using find to find the start index
    # start index + length of gene to find the end index
    indices = []
    count = 0
    index = variant.find(gene)
    # print(f'index before entering while loop: {index}')
    while index >= 0:
        # print(f'entered while loop')
        indices.append((index+1, index+len(gene)+1))  # bp locations of genes reported by databases start from 1
        count += 1
        if count % 10:
            print(f'found at index {index}, len(variant): {len(variant)}')
        index = variant.find(gene, index + len(gene))
    return indices

gene = shmt
variant = williams_sequence
gene_boundaries = find_start_end(gene, variant)
print(f'boundaries of Forrest SHMT gene in Williams sequence: {gene_boundaries}')

boundaries of Forrest SHMT gene in Williams sequence: []
